### boilerplate for snippets

In [1]:
// general includes
#include <iostream>   // std::cout|endl 
#include <vector>   // std::vector
using namespace std;

In [2]:
namespace iue {
struct Widget {
    double d;
};
}

In [3]:
using namespace iue;

## Quick Recap



- Types and value categories
    - value types
        ```c++
        void func(Widget value); // creates a copy
        ```
    - reference types
        ```c++
        void func(const Widget& ref); // binds to rvalue or lvalue
        ```    
    - (pointer types)
        ```c++
        void func(const Widget* ptr); // pointer is copied
        ```        
- Binding rules (for references): 
    - cannot bind if constness is lost
    - cannot bind rvalue to non-const reference
    - const reference can bind to rvalues, too

## Goals for today


- Understanding/intuition for the function overloading mechanism in C++.
- Understanding/intuition for classes holding dynamic memory (via a raw pointer) under the hood.
- Understand why *move semantics* where added with C++11 and which problem is solved with it.

#  Overload resolution

**Simple example of a function overload?**

```c++
void update(Other& w);
void update(Widget& w);
```

**How is overloading done *behind the scenes*?**
- example C: `func.c`
- example C++: `func.cpp`


According to detailed rules the compiler performs a lookup for the *name* of a function 
- in the scope of the call site (*name lookup*) [(cppref)](https://en.cppreference.com/w/cpp/language/lookup) followed by a lookup 
- in the scopes of the arguments (*argument dependent lookup*) [(cppref)](https://en.cppreference.com/w/cpp/language/adl).

The rules are quite lengthy, and there is no urgent need to know all details. 
Finally, a set of *candidate functions* is present, which enter the step of *overload resolution* [(cppref)](https://en.cppreference.com/w/cpp/language/overload_resolution) with the aim to find the "best fitting" function to a function call expression.

# Holder classes

We can use `std::vector` and `std::string` as two prominent example for classes with a dynamic size of elements.
Both use a dynamically allocated memory, which means a raw pointer to this memory is hold. 

In [4]:
template <typename T>
class std_vector_mock {
    T* _data; 
    size_t _n;
    // ...
};
template <typename CHAR_T>
class std_string_mock {
    CHAR_T* _data; 
    size_t _n;
    // ...
};

**How is such a holder class copied?**

**How and when is the dynamic memory allocated?**

**How and when is the dynamic memory deallocated?**

# Move semantics

Consider the following example with a Holder class which holds two members, each relying on dynamic memory allocations to store the data:

In [5]:
struct Holder {
    std::vector<Widget> data;  // holds dynamic memory
    std::string name;          // holds dynamic memory 
};

**How to write a function which constructs a Holder from a vector and a string, efficiently?**


C++11 introduced a new kind of reference termed **rvalue reference** which takes part in overload resolution and enables *move semantics* via the following rules.

**Rule**: Rvalue references can bind to rvalues only.

**Another rule**: When binding to an rvalue reference (as function parameter) one **does** expect that the bound object will have "it's contents stolen", i.e. the internally allocated dynamic memory will be used elsewhere (in this example for the construction of the members of the returned `Holder`), and the state of the objects will be 
- "valid but empty" or 
- "valid and filled with something else"
after the function call.

In [6]:
Holder init(std::vector<Widget> &&data, std::string  &&name){
    return Holder{std::move(data),std::move(name)};
}

As move semantics are an extension to an existing language specification, this new language feature does rely on a utility function from the standard library: `std::move`.


## std::move 

When calling a function with an rvalue-reference parameter, there are two options for binding:



**Option1**: Pass rvalues (temporaries) directly


In [7]:
Holder hold1 = init(std::vector<Widget>{{1.0},{1.0}}, "literal");  // pass temporaries

**Option2**: Explicitly cast an lvalue to an rvalue using `std::move` [(cppref)](https://en.cppreference.com/w/cpp/utility/move)

In [8]:
auto vec = std::vector<Widget>(2,Widget{1});    // (1) lvalue variable
std::string str = "literal";                    // (2) lvalue variable

// Holder hold2 = init(vec,str);  // pass lvalues        
Holder hold3 = init(std::move(vec), std::move(str)); // pass explicitly move lvalues 

// state of 'vec' and 'str' here?

**What are the guarantees for the caller for the state of `vec` and `str` after moving?**

### Overload resolution rules for rvalue parameters
Overload resolution rules also considers the rvalue-ness of arguments when determining which function fits best. Consider the following two functions:

In [13]:
Holder init(const std::vector<Widget> &data, const std::string &name){
    return Holder{data,name};
}

In [10]:
Holder init(std::vector<Widget> &&data, std::string &&name){
    return Holder{std::move(data),std::move(name)};
}           

Both functions can bind to rvalues, but in this case the "real-rvalues" win is called when rvalue arguments are passed.

### Detail: New value categories

C++11 introducesa new taxonomy for value categories [(cppref)](https://en.cppreference.com/w/cpp/language/value_category):

- *locatable* values: glvalues = lvalues AND **xvalues** 
- *movable* values: rvalues = **xvalues** AND **prvalues**

![(images/value_categories.svg)](images/value_categories.svg)


In [11]:
// xvalues: location: yes, movable: yes
Holder hold4 = init(std::move(vec), std::move(str));
//                  ^^^^^^^^^^^^^^  ^^^^^^^^^^^^^^
//                  xvalue expr.    xvalue expr.

// prvalues: location: no, movable: yes     
Holder hold5 = init(std::vector<Widget>{{1.0}}, std::string{"literal"}); 
//                  ^^^^^^^^^^^^^^^^^^^^^^^^^^  ^^^^^^^^^^^^^^^^^^^^^^
//                    prvalue expr.               prvalue expr.

# Summary

- overload resolution considers all arguments **and** also distinguishes between rvalue and lvalue categories
- move semantics allow for a very desirable option: efficient composition ("factory") patterns

```c++
void init(Widget value); // creates a copy
void init(const Widget& ref); // binds to rvalue (non-moving) or lvalue
void init(Widget&& ref); // binds to rvalue only (moving)
```